In [1]:
import pandas as pd
import glob
import os
import json
import concurrent.futures
from multiprocessing import Pool

In [2]:
f = open('config.json', "r") 
data = json.loads(f.read())['OptionsTickDataAggregator'] 
f.close() 

In [3]:
def getSubDir(m_path):
    return [os.path.basename(x[0]) for x in os.walk(m_path)][1:]

In [4]:
def fileToDf(m_path):

    all_df = []

    for n in sub_dirs:
        i_path = os.path.join(m_path , n)
        all_files = glob.glob(i_path + "/*.csv")
        li = []
        for filename in all_files:
            df = pd.read_csv(filename, index_col=None, header=0)
            names = os.path.splitext(os.path.basename(filename))[0].split()
            df.insert(0,'StrikePrice', int(float(names[5])), True)
            df.insert(0,'OptionType', names[4],True)
            df.insert(0,'ExpiryDate', f'{names[1]}-{names[2]}-{names[3]}', True)
            df.insert(0,'Symbol',names[0],True)

            li.append(df)

        frame = pd.concat(li, axis=0, ignore_index=True)
        frame = frame.applymap(str)
        all_df.append(frame)
        
    return all_df

In [5]:
def mergeFiles(sd,ed):
    
    frames = fileToDf(dir_path)
    
    for n, frame  in enumerate(frames):
        dt = sub_dirs[n]
        if int(sd) <= int(dt) <= int(ed):
            opath = os.path.join(data['OutputDirectory'], dt)
            frame.to_csv(f'{opath}.csv' , encoding='utf-8', index=False)


In [6]:
if __name__ == "__main__":
    try:  
        os.mkdir(data['OutputDirectory']) 
    except Exception as error:  
        print(error)  
      
    pool = Pool(processes=int(data['Processors']))
    
    dir_path = data['InputDirectory']
    sub_dirs = getSubDir(dir_path)
    
    sd = data['StartDate']
    ed = data['EndDate']

    mergeFiles(sd,ed)
    
    pool.close()

[WinError 183] Cannot create a file when that file already exists: 'C:/Users/smrnj/Desktop/PyTest/OutputDirectory/'


In [7]:
df = pd.read_csv('C:/Users/smrnj/Desktop/PyTest/OutputDirectory/20191104.csv')

df

,Symbol,ExpiryDate,OptionType,StrikePrice,Timestamp,Last,LastSize,TotalVolume,Bid,Ask,TickId,BasisForLast,TradeMarketCenter,TradeConditions
0,AMKR,20-MAR-20,PUT,11,0.53,1,1,0.50,0.55,63857359,C,9,01,NaN
1,AMKR,20-MAR-20,PUT,11,0.55,3,4,0.50,0.55,64031460,C,156,3D,NaN
2,AMKR,20-MAR-20,PUT,11,0.50,1,5,0.50,0.55,74611017,C,11,20,NaN
3,AMKR,20-MAR-20,PUT,14,1.75,1,1,1.70,1.75,20621700,C,17,20,NaN
4,AMLP,08-NOV-19,PUT,9,0.30,10,10,0.15,0.45,3672704,C,9,20,NaN
5,AMLP,13-DEC-19,CALL,8,0.35,22,22,0.25,0.35,20585047,C,88,3D,NaN
6,AMLP,13-DEC-19,CALL,8,0.35,8,30,0.25,0.35,20585048,C,88,3D,NaN
7,AMLP,13-DEC-19,CALL,8,0.35,3,33,0.25,0.35,20585048,C,88,3D,NaN
8,AMLP,13-DEC-19,CALL,8,0.35,1,34,0.25,0.35,20585049,C,88,3D,NaN
9,AMLP,13-DEC-19,CALL,8,0.35,1,35,0.25,0.35,20585050,C,88,3D,NaN
